# Face Mask Detection

In [1]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np

In [2]:
model = tf.keras.models.load_model('trained_model')

In [3]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [4]:
labels = {0:' WITH MASK', 1:'WITHOUT MASK'}
colors = {0:(0,255,0),1:(0,0,255)}

In [5]:
source = cv2.VideoCapture(0) # 0 is defeault camera

while True:
    ret, frame = source.read()
    if ret==True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face_coordinations = face_clsfr.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=8,
        minSize=(10, 10))
        
        for (x,y,w,h) in face_coordinations:
            img = frame[y:y+h,x:x+w,:]
            resized = cv2.resize(img,(224, 224))
            normalized = resized/255.0
            reshaped = np.reshape(normalized,(1,224, 224, 3))
            out = model.predict(reshaped)
            label = np.argmax(out)
            cv2.rectangle(frame, (x,y),(x+w,y+h), colors[label],3)
            cv2.rectangle(frame, (x,y-35),(x+w,y), colors[label],-1)
            cv2.putText(frame, labels[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX ,0.85,(255,255,255),2)
        #to stop the program press q
        cv2.imshow('facemask-detection',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break
        
cv2.destroyAllWindows()
source.release()